In [1]:
from collections import Counter
import random
import copy

In [2]:
class Char(int):
    # Return an inverse of the char
    def inv(self):
        return Char(-self)
    
    def letter(self):
        return Char(abs(self))
    
    # Check whether the specified `Char` is an inverse of the current
    def is_inv(self, other_char) -> bool:
        return other_char + self == 0
    
    def __str__(self):
        if self > 0:
            return f"x{int(self)}"
        else:
            return f"X{-int(self)}"

In [3]:
class Word(list[Char]):
    # Create a copy of the word
    def clone(self):
        return Word(copy.deepcopy(self))

    def inv(self):
        return Word([e.inv() for e in reversed(self)])
    
    def shift(self, i: int):
        return Word(self[i:] + self[:i])
    
    def cyclic_shifts(self):
        for i in range(len(self)):
            yield Word(self[i:] + self[:i])

    # Reduce the word
    def reduce(self):
        char_stack: list[Char] = []
        for char in self:
            if len(char_stack) == 0:
                char_stack.append(char)
                continue

            if not char_stack[-1].is_inv(char):
                char_stack.append(char)
            else:
                char_stack.pop()
        self.clear()
        self.extend(char_stack)

    # Cyclically redude the word
    def reduce_cycl(self):
        self.reduce()
        # Pop front and end while they cancel each other
        while len(self) > 0 and self[0].is_inv(self[-1]):
            self.pop(0)
            self.pop()

    # Substitute char in word
    def substitute(self, char: Char, word):
        from itertools import chain
        def recalculate(other: Char) -> list[Char]:
            if char == other:
                return word.clone()
            if char.inv() == other:
                return word.inv()
            return [other]
        self_clone = self.clone()
        self.clear()
        self.extend(chain(*[recalculate(e) for e in self_clone]))
        self.reduce_cycl()

    # String representation of the word
    def __str__(self):
        return " ".join(map(lambda c: str(c), self))

In [4]:
def gen_sudoku(xs, ys):
    while True:
        lookup_table = [(x, y) for x in range(1, xs+1) for y in range(1, ys+1)]
        res = []
        # error = False
        try:
            for x in range(1, xs+1):
                for y in range(1, ys+1):
                    if (x, y) not in lookup_table:
                        continue
                    lookup_table.remove((x, y))
                    random.shuffle(lookup_table)

                    def check(x1, y1, x2, y2):
                        #return True
                        if x1 == x2 or y1 == y2:
                            return False
                        for xx1, yy1, xx2, yy2 in res:
                            # if x1 == xx1 and x2 == xx2:
                            #     return False
                            # if x1 == xx2 and x2 == xx1:
                            #     return False
                            if y1 == yy1 and y2 == yy2:
                                return False
                            if y1 == yy2 and y2 == yy1:
                                return False
                            pass
                        return True
                    candidate_x, candidate_y = next((xx, yy) for (xx, yy) in lookup_table if check(x, y, xx, yy))
                    lookup_table.remove((candidate_x, candidate_y))
                    res += [(x, y, candidate_x, candidate_y)]
            vocabulary = {Char(t): Word([Char(t)]) for t in range(1, xs + ys + 1)}
            ans = [Word(map(Char, [x1, xs + y1, - xs -y2, -x2])) for x1, y1, x2, y2 in res]
            # return ans, vocabulary
            return ans
        except:
            continue

In [5]:
gen_sudoku(5, 6)
# interpret it as x_1y_6 = x_3y_10

[[1, 6, -11, -3],
 [1, 7, -9, -3],
 [1, 8, -11, -4],
 [1, 9, -10, -3],
 [1, 10, -6, -3],
 [1, 11, -10, -2],
 [2, 6, -7, -5],
 [2, 7, -8, -4],
 [2, 8, -9, -5],
 [2, 9, -6, -4],
 [2, 11, -7, -3],
 [3, 8, -6, -5],
 [4, 7, -10, -5],
 [4, 9, -11, -5],
 [4, 10, -8, -5]]

In [7]:
import numpy as np
from numpy.linalg import matrix_rank

m = np.matrix([[1, 2], [3, 4]])

ModuleNotFoundError: No module named 'numpy'

In [10]:
def make_matrix(xs, ys, ls): 
    num_generators = xs + ys
    num_relators = int(xs * ys * 0.5)
    
    out_matrix = matrix([[0 for _ in range(0, num_relators)] for _ in range(0, num_generators)])
    
    for ind, l in enumerate(ls): 
        x1 = l[0]
        y1 = l[1]
        x2 = -l[3]
        y2 = -l[2]
        # relation: x_1 + y_1 + x_2 + x_3 = 0
        out_matrix[x1-1][ind] = 1
        out_matrix[x2-1][ind] = -1
        out_matrix[xs + y1-1][ind] = 1
        out_matrix[xs + y2-1][ind] = -1
    return out_matrix


make_matrix(5, 6, gen_sudoku(5, 6))        

NameError: name 'matrix' is not defined